In [1]:
import sys
import os
sys.path.append('../')
import data_readers

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import math

# networkx
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

# For semantic similarity
from urllib.parse import unquote
from sentence_transformers import SentenceTransformer
import torch

# Python functions in .py file to read data
import machine_searchers
import time

import warnings
from tqdm import TqdmWarning
warnings.filterwarnings('ignore', category=TqdmWarning)

In [4]:
df = pd.read_csv('machine_data_runs_18000_269.csv')
display(df)

,Unnamed: 0,Index,Source,Target,Length_1,Path_1,Explored_1,Length_2,Path_2,Explored_2
0,18000,18000,Mustard plant,Sydney Newman,2925,"['Mustard plant', 'United States', 'Publishing...","{'Mustard plant': None, 'Biodiesel': 'Mustard ...",80,[],"{'Mustard plant': None, 'Biodiesel': 'Mustard ..."
1,18001,18001,Mustard plant,Ward Cunningham,1426,"['Mustard plant', 'United States', 'Publishing...","{'Mustard plant': None, 'Biodiesel': 'Mustard ...",123,[],"{'Mustard plant': None, 'Biodiesel': 'Mustard ..."
2,18002,18002,Mustard plant,Yugoslavia,11,"['Mustard plant', 'Canada', 'Yugoslavia']","{'Mustard plant': None, 'Biodiesel': 'Mustard ...",76,[],"{'Mustard plant': None, 'Biodiesel': 'Mustard ..."
3,18003,18003,Mustard seed,Albert Einstein,116,"['Mustard seed', 'Milk', 'Milky Way', 'Albert ...","{'Mustard seed': None, 'Cymbopogon': 'Mustard ...",34,"['Mustard seed', 'Cymbopogon', 'Thailand', 'Li...","{'Mustard seed': None, 'Cymbopogon': 'Mustard ..."
4,18004,18004,Mustard seed,Canada,22,"['Mustard seed', 'Mustard plant', 'Canada']","{'Mustard seed': None, 'Star anise': 'Mustard ...",86,"['Mustard seed', 'Star anise', 'Binomial nomen...","{'Mustard seed': None, 'Star anise': 'Mustard ..."
...,...,...,...,...,...,...,...,...,...,...
514,18514,18514,Nigeria,Quartz,494,"['Nigeria', 'California', 'San Jose, Californi...","{'Nigeria': None, 'List of countries by system...",182,[],"{'Nigeria': None, 'List of countries by system..."
515,18515,18515,Nigeria,Romeo and Juliet,476,"['Nigeria', 'United Kingdom', 'Arctic Monkeys'...","{'Nigeria': None, 'World Health Organization':...",54,[],"{'Nigeria': None, 'World Health Organization':..."
516,18516,18516,Nigeria,Scotland,16,"['Nigeria', 'Petroleum', 'Scotland']","{'Nigeria': None, 'Natural gas': 'Nigeria', 'C...",59,"['Nigeria', 'Natural gas', 'Phase (matter)', '...","{'Nigeria': None, 'Natural gas': 'Nigeria', 'P..."
517,18517,18517,Nigeria,The Ashes,195,"['Nigeria', 'London', 'Trafalgar Square', 'The...","{'Nigeria': None, 'List of countries by system...",69,[],"{'Nigeria': None, 'List of countries by system..."


In [8]:
df.loc[0]['Path_1']

"['Mustard plant', 'United States', 'Publishing', 'Doctor Who', 'Sydney Newman']"

In [12]:
import ast
s = df.loc[0]['Path_1']
lst = ast.literal_eval(s)
# Output: ['apple', 'banana', 'cherry']
print(lst)


['Mustard plant', 'United States', 'Publishing', 'Doctor Who', 'Sydney Newman']


In [13]:
df['Path_1'] = df['Path_1'].apply(lambda x: ast.literal_eval(x))
display(df)

,Unnamed: 0,Index,Source,Target,Length_1,Path_1,Explored_1,Length_2,Path_2,Explored_2
0,18000,18000,Mustard plant,Sydney Newman,2925,"[Mustard plant, United States, Publishing, Doc...","{'Mustard plant': None, 'Biodiesel': 'Mustard ...",80,[],"{'Mustard plant': None, 'Biodiesel': 'Mustard ..."
1,18001,18001,Mustard plant,Ward Cunningham,1426,"[Mustard plant, United States, Publishing, Wik...","{'Mustard plant': None, 'Biodiesel': 'Mustard ...",123,[],"{'Mustard plant': None, 'Biodiesel': 'Mustard ..."
2,18002,18002,Mustard plant,Yugoslavia,11,"[Mustard plant, Canada, Yugoslavia]","{'Mustard plant': None, 'Biodiesel': 'Mustard ...",76,[],"{'Mustard plant': None, 'Biodiesel': 'Mustard ..."
3,18003,18003,Mustard seed,Albert Einstein,116,"[Mustard seed, Milk, Milky Way, Albert Einstein]","{'Mustard seed': None, 'Cymbopogon': 'Mustard ...",34,"['Mustard seed', 'Cymbopogon', 'Thailand', 'Li...","{'Mustard seed': None, 'Cymbopogon': 'Mustard ..."
4,18004,18004,Mustard seed,Canada,22,"[Mustard seed, Mustard plant, Canada]","{'Mustard seed': None, 'Star anise': 'Mustard ...",86,"['Mustard seed', 'Star anise', 'Binomial nomen...","{'Mustard seed': None, 'Star anise': 'Mustard ..."
...,...,...,...,...,...,...,...,...,...,...
514,18514,18514,Nigeria,Quartz,494,"[Nigeria, California, San Jose, California, Qu...","{'Nigeria': None, 'List of countries by system...",182,[],"{'Nigeria': None, 'List of countries by system..."
515,18515,18515,Nigeria,Romeo and Juliet,476,"[Nigeria, United Kingdom, Arctic Monkeys, Rome...","{'Nigeria': None, 'World Health Organization':...",54,[],"{'Nigeria': None, 'World Health Organization':..."
516,18516,18516,Nigeria,Scotland,16,"[Nigeria, Petroleum, Scotland]","{'Nigeria': None, 'Natural gas': 'Nigeria', 'C...",59,"['Nigeria', 'Natural gas', 'Phase (matter)', '...","{'Nigeria': None, 'Natural gas': 'Nigeria', 'P..."
517,18517,18517,Nigeria,The Ashes,195,"[Nigeria, London, Trafalgar Square, The Ashes]","{'Nigeria': None, 'List of countries by system...",69,[],"{'Nigeria': None, 'List of countries by system..."


In [ ]:
import networkx as nx
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assuming you have a list of edges representing hyperlinks between articles
edges = [('Article1', 'Article2'), ('Article2', 'Article3'), ...]

# Create a directed graph from the edges
G = nx.DiGraph(edges)

# Calculate network metrics for each article
centrality = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
pagerank = nx.pagerank(G)

# Combine the metrics into a DataFrame
metrics_df = pd.DataFrame({
    'Article': list(G.nodes),
    'Betweenness': [centrality[article] for article in G.nodes],
    'Closeness': [closeness[article] for article in G.nodes],
    'PageRank': [pagerank[article] for article in G.nodes]
})

# Now, let's say you have a DataFrame of target articles and a DataFrame of potential matches
# Both DataFrames need to have the same structure and include the network metrics calculated above

# Example DataFrames (you would have actual data here)
target_articles_df = pd.DataFrame({
    'Article': ['Target1', 'Target2', ...],
    # Include other relevant article features here
    'Betweenness': [...],
    'Closeness': [...],
    'PageRank': [...]
})

potential_matches_df = pd.DataFrame({
    'Article': ['Potential1', 'Potential2', ...],
    # Include other relevant article features here
    'Betweenness': [...],
    'Closeness': [...],
    'PageRank': [...]
})

# Use NearestNeighbors to find the closest match in the potential matches for each target article
# This is using Euclidean distance but you can use other distances as appropriate

# Fit the nearest neighbors model to the potential matches
nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
nn.fit(potential_matches_df[['Betweenness', 'Closeness', 'PageRank']])

# Find the nearest neighbor for each target article
distances, indices = nn.kneighbors(target_articles_df[['Betweenness', 'Closeness', 'PageRank']])

# Add the index of the nearest neighbor to the target articles DataFrame
target_articles_df['MatchIndex'] = indices.flatten()

# Add the matched article information to the target articles DataFrame
target_articles_df['MatchedArticle'] = potential_matches_df.iloc[target_articles_df['MatchIndex']]['Article'].values


In [ ]:
# Merge the matched pairs
treated['match_index'] = indices.flatten()
matched_pairs = treated.merge(control, left_on='match_index', right_index=True, suffixes=('_treated', '_control'))

# Perform a paired t-test to compare the path lengths
from scipy.stats import ttest_rel
ttest_rel(matched_pairs['path_length_treated'], matched_pairs['path_length_control'])


In [ ]:
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant
from sklearn.neighbors import NearestNeighbors

# Assume df is a DataFrame containing your data, including 'path_length', 'category', and your covariates

# Calculate the propensity scores
df_with_const = add_constant(df[covariates])  # Add constant for the logistic regression
logit_model = Logit(df['treatment'], df_with_const).fit()
df['propensity_score'] = logit_model.predict(df_with_const)

# Match treatment and control units
treated = df[df['treatment'] == 1]
control = df[df['treatment'] == 0]

# Use Nearest Neighbors for matching on the propensity score
nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(control[['propensity_score']])
distances, indices = nn.kneighbors(treated[['propensity_score']])


In [ ]:
# Reset the index to avoid errors in the graph construction
acl_reviews.reset_index(inplace=True) 

# Separate the treatment and control groups
treatment_df = acl_reviews[acl_reviews['had_rebuttal_int'] == 1]
control_df = acl_reviews[acl_reviews['had_rebuttal_int'] == 0]

# Create an empty undirected graph
G = nx.Graph()

# Loop through all the pairs of instances
for control_id, control_row in control_df.iterrows():
    for treatment_id, treatment_row in treatment_df.iterrows():
         # Given the instruction of the exact matching, we only add an edge if 
         # the track is the same and if the overall score is the same
        if control_row['track'] == treatment_row['track']:
            if control_row['overall_score_before_avg'] == treatment_row['overall_score_before_avg']:
                G.add_edge(control_id, treatment_id)
        

# Generate and return the maximal matching on the generated graph
matching = nx.maximal_matching(G)

# We create a list with all the indexes of the matched instances
matched = [i[0] for i in list(matching)] + [i[1] for i in list(matching)]
matched_df = acl_reviews.iloc[matched]